In [19]:
import matplotlib.pyplot as plt
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

import time

import pickle


In [2]:
def load_img_data():

    DATASTORE = 'C:\\Datasets\\bacterias'
    bacs = []
    dirs = os.listdir(DATASTORE)
    for i in dirs:
        dirpath = os.path.join(DATASTORE, i)
        files = os.listdir(dirpath)
        for j in files:
            if j == 'masked.pickle':
                pickle_in = open(os.path.join(dirpath, j), "rb")
                bacs.append(pickle.load(pickle_in))
    return bacs

def get_layer(hsi, layer):
    return hsi[:, :, layer]

def show_img_on_wave(cube, layer):
    mat = get_layer(cube, layer)
    plt.imshow(mat, cmap='gray')
    plt.show()


def hsi2mat(arr):
    if len(arr.shape) != 3:
        raise BaseException('A entrada deve possuir 3 dimensões')

    r, c, w = arr.shape
    return np.reshape(arr, (r*c, w))


def mat2hsi(mat, shape):
    return np.reshape(mat, shape)

def remove_blank_lines(bacs):
    new_bacs = []
    for i in bacs:
        bac = hsi2mat(i)
        new_bac = bac[~np.all(bac == 0, axis=1)]
        new_bacs.append(new_bac)
    return new_bacs

def create_training_dataset(bacs):
    trainingset = []
    for ind, val in enumerate(bacs):
        trainingset.append([val, ind])
    return separate_xy(trainingset)

def concat_lines(X, bac, cl):
    for i in range(bac.shape[0]):
        line = bac[i, :]
        line = np.append(line, cl)
        X.append(line)
    return X


def create_big_matrix(trainingset):
    X = []
    for f, l in trainingset:
        X = concat_lines(X, f, l)
    return X

def separate_xy(trainingset):
    mat = np.array(create_big_matrix(trainingset))
    np.random.shuffle(mat)
    print(mat.shape)
    X = mat[:, :241]
    y = mat[:, 241]
    return X,y

def load():
    bacs = load_img_data()
    nozeros = remove_blank_lines(bacs)
    return create_training_dataset(nozeros)


def normalize(data):
    return tf.keras.utils.normalize(data, 1)

In [3]:
X, y = load()
xx = normalize(X)


(725600, 242)


In [4]:
r,c = xx.shape
bb = np.array(xx).reshape(-1, r,c, 1)
print(bb.shape)


(1, 725600, 241, 1)


In [18]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)

sess = tf.compat.v1.Session(config=tf.ConfigProto(gpu_options=gpu_options))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


AttributeError: module 'tensorflow' has no attribute 'ConfigProto'

In [6]:
NAME = 'bacs-64x2-{}'.format(int(time.time()))

tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

In [10]:
model = Sequential()

model.add(Conv2D(16, 3, padding='same', input_shape=bb.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(16, 3))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(16))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, epochs=20, validation_split=0.3, callbacks=[tensorboard])


ResourceExhaustedError: OOM when allocating tensor with shape[696576000,16] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]